[Runnable Parallel : Manipulating Data](https://python.langchain.com/docs/expression_language/how_to/map)

In [5]:
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [3]:
# We will be using the Titan Embeddings Model to generate our Embeddings.
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock

# - create the Anthropic Model
llm = Bedrock(model_id="anthropic.claude-v2:1",  model_kwargs={'max_tokens_to_sample':200})
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1")

In [9]:


vectorstore = FAISS.from_texts(
    ["harrison worked at kensho","Tanay worked at Groupon"], embedding=bedrock_embeddings,
)
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context and say only where the person worked:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = llm

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("where did harrison work?")

' Based on the context provided, harrison worked at kensho.'

In [10]:
from operator import itemgetter

In [13]:
vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=bedrock_embeddings,
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": "where did harrison work", "language": "bengali"})

' হ্যারিসন কেনশো-তে কাজ করেছে।'

In [14]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel


model = llm
joke_chain = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
poem_chain = (
    ChatPromptTemplate.from_template("write a 2-line poem about {topic}") | model
)

map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

map_chain.invoke({"topic": "bear"})

{'joke': " Why don't bears like fast food? Because they can't catch it!",
 'poem': " Here's a 2-line poem about a bear:\n\nFurry and brown, the bear prowls around\nSeeking berries and fish to be found"}

### Parallelism
RunnableParallel are also useful for running independent processes in parallel, since each Runnable in the map is executed in parallel. For example, we can see our earlier joke_chain, poem_chain and map_chain all have about the same runtime, even though map_chain executes both of the other two.

In [15]:
%%timeit

joke_chain.invoke({"topic": "bear"})

1.66 s ± 746 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%%timeit

poem_chain.invoke({"topic": "bear"})

2.17 s ± 851 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%%timeit

map_chain.invoke({"topic": "bear"})

The slowest run took 8.98 times longer than the fastest. This could mean that an intermediate result is being cached.
4.11 s ± 4.19 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
